<a href="https://colab.research.google.com/github/Anushreesharma23/AI-Voice-Assistance/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp pypdf
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -q install llama-cpp-python

In [ ]:
import os
import time

from llama_index.core import Prompt, StorageContext, load_index_from_storage, Settings, VectorStoreIndex, SimpleDirectoryReader, set_global_tokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

from transformers import AutoTokenizer

In [ ]:
pdf_directory= '/content/sample_data/data'
text_embedding_model = 'thenlper/gte-base'
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'
# set_global_tokenizer(AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode)

In [ ]:
from google.colab import files

# Upload files to the specified directory
uploaded = files.upload()

# Move the files to the desired directory
import os
os.makedirs("/content/sample_data/data", exist_ok=True)
for filename in uploaded.keys():
    os.rename(filename, f"/content/sample_data/data/{filename}")

print("Files moved to /content/sample_data/data")

Saving AnushreeSharma Resume.pdf to AnushreeSharma Resume.pdf
Saving AnushreeSharmaResume (2).pdf to AnushreeSharmaResume (2).pdf
Saving car.pdf to car.pdf
Saving chair.pdf to chair.pdf
Files moved to /content/sample_data/data


In [ ]:
# Set the directory containing the PDF files
pdf_directory = "/content/sample_data/data"

# List all files in the directory to verify
print("Files in directory:", os.listdir(pdf_directory))

Files in directory: ['File-07ewW.pdf', 'File-nCxVC.pdf', 'File-7nZvz.pdf', 'File-ghDEp.pdf', 'File-L9NA4.pdf', 'File-QO2IF.pdf', 'File-7XXbn.pdf', 'File-2Y8tw.pdf', 'File-UDsqT.pdf', 'car.pdf', 'File-qCvaB.pdf', 'Documents AnushreeSharma.pdf', 'File-S0F4k.pdf', 'File-Rv9ZX.pdf', 'File-3I0jK.pdf', 'File-Y1Ctu.pdf', 'File-3Fnwy.pdf', 'File-xDhNj.pdf', 'AnushreeSharmaResumeSDE.pdf', 'File-gsmGZ.pdf', 'File-YmJS9.pdf', 'Flowers.pdf', 'File-eGPRo.pdf', 'File-MDnHB.pdf', 'File-loPbc.pdf', 'File-zm4aY.pdf', 'AnushreeSharmaResume (2).pdf', 'cover letter GS.pdf', 'File-TiVxa.pdf', 'File-C9xjg.pdf', 'File-6RaXb.pdf', 'File-jmwGj.pdf', 'File-lxqu9.pdf', 'File-lUpvR.pdf', 'File-K2Onb.pdf', 'File-oE5EM.pdf', 'File-RlQkI.pdf', 'File-rRrLy.pdf', 'File-ffdNi.pdf', 'fan.pdf', 'File-waJh6.pdf', 'File-Su5M5.pdf', 'File-vDn3p.pdf', 'Appointment Reciept.pdf', 'File-q0nan.pdf', 'File-TEEIx.pdf', 'File-mjcIs.pdf', 'File-jb0Vn.pdf', 'AnushreeSharma Resume.pdf', 'File-7G7lc.pdf', 'ielts.pdf', 'File-twwuu.pdf',

In [ ]:
# Set the directory containing the PDF files
pdf_directory = "/content/sample_data"

# List all files in the directory to verify
print("Files in directory:", os.listdir(pdf_directory))

Files in directory: ['README.md', 'anscombe.json', '.ipynb_checkpoints', 'data', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'california_housing_test.csv']


In [ ]:
# Load PDF
filename_fn = lambda filename: {'file_name': os.path.basename(pdf_directory)}
loader = SimpleDirectoryReader(input_dir = pdf_directory, recursive=True)
documents = loader.load_data(num_workers=3)

In [ ]:
# Load models and service context
start_time = time.time()
embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
llm = LlamaCPP(model_url=llm_url, temperature=0.5, max_new_tokens=256, context_window=4096, generate_kwargs = {"stop": ["<s>", "[INST]", "[/INST]"]}, model_kwargs={"n_gpu_layers": -1}, verbose=True)
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model, chunk_size=512)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
end_time = time.time()
index_creation_time = end_time - start_time
print(f"Time taken to create the index: {index_creation_time:.2f} seconds")

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_c

Time taken to create the index: 4.01 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: llama-2


In [ ]:
start_time = time.time()
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
end_time = time.time()
index_creation_time = end_time - start_time
print(f"Time taken to create the index: {index_creation_time:.2f} seconds")

Time taken to create the index: 2.25 seconds


In [ ]:
# Prompt Template (RAG)
concise_template = Prompt("""
Based on the following context from the document, providing a concise response to the user's query:
Document Context:
{context_str}
User Query:
{query_str}

Response (one paragraph):
""")

# Query Engine
query_engine = index.as_query_engine(streaming=True, llm=llm)


In [ ]:
# Inferencing with RAG
conversation_history = "Hi. [INST] How can I assist you with the content of the document today?"

def clean_text(text):
    # Remove <pad> tokens and unnecessary newlines
    cleaned_text = text.replace("<pad>", "").replace("<EOS>", "").replace("\n", " ")
    # Remove extra spaces
    cleaned_text = " ".join(cleaned_text.split())
    return cleaned_text

while True:
    user_query = input("User: ")
    if user_query.lower() == "exit":
        break
    conversation_history += user_query + " [/INST] "

    start_time = time.time()

    response = query_engine.query(user_query)
    print("\nDocument Context Retrieved:")
    for source_document in response.source_nodes:
        cleanText = clean_text(source_document.node.get_text())
        print(cleanText)

    print("\nDocument Context Retrieved:")
    for source_document in response.source_nodes:
        print(source_document.node.get_text())

User: what is tree?

Document Context Retrieved:
A tree is a perennial plant with an elongated stem, or trunk, supporting branches and leaves. Trees are an essential part of the ecosystem, providing oxygen, improving air quality, conserving water, preserving soil, and supporting wildlife. They also offer shade and beauty to human environments.
Flowers are the reproductive structures of flowering plants, typically featuring colorful petals and distinct shapes. They play a vital role in the reproduction of plants by facilitating pollination, which insects, birds, or wind often aid . Flowers can vary greatly in size, color, and fragrance, attracting different pollinators. Beyond their biological importance, flowers are cherished for their beauty and are often used in decorations, ceremonies, and as symbols of affection.

Document Context Retrieved:
A tree is a perennial plant with an elongated stem, or trunk, supporting branches and leaves. Trees 
are an essential part of the ecosystem, p

In [ ]:
# Define a function to filter context based on a specified keyword
def filter_context(context, start_keyword):
    start_index = context.find(start_keyword)
    if start_index != -1:
        return context[start_index:]
    return context  # Return the original context if the keyword is not found

# Define a function to clean and filter text
def clean_text(text):
    cleaned_text = text.replace("<pad>", "").replace("<EOS>", "").replace("\n", " ")
    cleaned_text = " ".join(cleaned_text.split())
    cleaned_text = filter_context(cleaned_text, "Figure 3")
    return cleaned_text

# Inferencing with RAG
conversation_history = "Hi. [INST] How can I assist you with the content of the document today?"

while True:
    user_query = input("User: ")
    if user_query.lower() == "exit":
        break
    conversation_history += user_query + " [/INST] "

    response = query_engine.query(user_query)
    print("\nDocument Context Retrieved:")
    print("The answer to your query retrieved from the document is:")
    for source_document in response.source_nodes:
        cleanText = clean_text(source_document.node.get_text())
        print(cleanText)

User: what is tree?

Document Context Retrieved:
The answer to your query retrieved from the document is:
A tree is a perennial plant with an elongated stem, or trunk, supporting branches and leaves. Trees are an essential part of the ecosystem, providing oxygen, improving air quality, conserving water, preserving soil, and supporting wildlife. They also offer shade and beauty to human environments.
Flowers are the reproductive structures of flowering plants, typically featuring colorful petals and distinct shapes. They play a vital role in the reproduction of plants by facilitating pollination, which insects, birds, or wind often aid . Flowers can vary greatly in size, color, and fragrance, attracting different pollinators. Beyond their biological importance, flowers are cherished for their beauty and are often used in decorations, ceremonies, and as symbols of affection.
User: what is chair?

Document Context Retrieved:
The answer to your query retrieved from the document is:
A chair

Streamlit application

In [1]:
# Step 1: Install necessary libraries
!pip install streamlit pyngrok llama_index
!pip install llama_index.embeddings.huggingface
!pip install llama_index.llms.llama_cpp
!pip install transformers
!pip install ngrok pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#/* Custom CSS for Streamlit app */
%%writefile style.css
body {
    background-color: #f0f2f6;
    color: #333;
    font-family: 'Arial', sans-serif;
}

header {
    background-color: #4CAF50;
    padding: 20px;
    text-align: center;
    font-size: 28px;
    color: white;
    border-radius: 8px;
    margin-bottom: 20px;
}

footer {
    background-color: #4CAF50;
    padding: 15px;
    text-align: center;
    font-size: 50px;
    color: white;
    position: relative;
    bottom: 0;
    width: 100%;
    border-radius: 8px 8px 0 0;
    margin-top: 20px;
}

h1, h2, h3, h4, h5, h6 {
    color: #4CAF50;
}

.stButton>button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 12px 24px;
    font-size: 16px;
    cursor: pointer;
    border-radius: 4px;
    transition: background-color 0.3s ease;
}

.stButton>button:hover {
    background-color: #45a049;
}

.stTextInput>div>div>input {
    padding: 10px;
    border: 2px solid #4CAF50;
    border-radius: 4px;
}

.stFileUploader>div>div>div>button {
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 10px 20px;
    font-size: 16px;
    cursor: pointer;
    border-radius: 4px;
    transition: background-color 0.3s ease;
}

.stFileUploader>div>div>div>button:hover {
    background-color: #45a049;
}

.stAlert {
    border-left: 5px solid #4CAF50;
    padding: 10px;
    border-radius: 4px;
}

.stSuccess {
    background-color: #e7f6e5;
    border-left: 5px solid #4CAF50;
    padding: 10px;
    border-radius: 4px;
}


Writing style.css


In [3]:
!ngrok config add-authtoken '2j9MNrMbeZEpvOcfinlFoOaatLS_7qwJiUjjX4P1tPGgrcb7P'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
%%writefile app.py
import os
import time
import streamlit as st
from google.colab import files
from llama_index.core import Prompt, Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from pyngrok import ngrok

# Define the upload and processing function
def upload_and_process_file():
    uploaded_file = st.file_uploader("Upload a PDF file", type="pdf")
    if uploaded_file is not None:
        # Save the uploaded file to a directory
        pdf_directory = "/content/sample_data/data"
        if not os.path.exists(pdf_directory):
            os.makedirs(pdf_directory)

        # Save the file to the directory
        file_path = os.path.join(pdf_directory, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Print the file path for debugging
        st.write(f"File saved to: {file_path}")

        # Process the PDF file
        try:
            index, llm = load_and_process_pdf(pdf_directory, text_embedding_model, llm_url)
            st.write("File processed successfully.")
        except ValueError as e:
            st.error(f"Error processing file: {e}")
    else:
        st.info("Please upload a PDF file.")

# Function to load and process PDF
def load_and_process_pdf(pdf_directory, text_embedding_model, llm_url):
    filename_fn = lambda filename: {'file_name': os.path.basename(pdf_directory)}
    loader = SimpleDirectoryReader(input_dir=pdf_directory, recursive=True)
    documents = loader.load_data(num_workers=3)

    embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
    llm = LlamaCPP(
        model_url=llm_url,
        temperature=0.5,
        max_new_tokens=256,
        context_window=4096,
        generate_kwargs={"stop": ["<s>", "[INST]", "[/INST]"]},
        model_kwargs={"n_gpu_layers": -1},
        verbose=True
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
    return index, llm

# Define a function to filter context based on a specified keyword
def filter_context(context, start_keyword):
    start_index = context.find(start_keyword)
    if start_index != -1:
        return context[start_index:]
    return context  # Return the original context if the keyword is not found

# Define a function to clean and filter text
def clean_text(text):
    cleaned_text = text.replace("<pad>", "").replace("<EOS>", "").replace("\n", " ")
    cleaned_text = " ".join(cleaned_text.split())
    cleaned_text = filter_context(cleaned_text, "Figure 3")
    return cleaned_text

# Load custom CSS
st.markdown('<style>' + open('style.css').read() + '</style>', unsafe_allow_html=True)

# Streamlit app
st.markdown('<h2>Document Retrieval using RAG</h2>', unsafe_allow_html=True)

pdf_directory = '/content/sample_data/data'
text_embedding_model = 'thenlper/gte-base'
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'

# Upload files
upload_and_process_file()

# Add a footer
st.markdown('<footer><p>Streamlit app by Anushree Sharma</p></footer>', unsafe_allow_html=True)


Writing app.py


In [11]:
# Step 3: Write the Streamlit app script
%%writefile app.py
import os
import time
import streamlit as st
from google.colab import files
from llama_index.core import Prompt, Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP

# Function to load and process PDF
def load_and_process_pdf(pdf_directory, text_embedding_model, llm_url):
    loader = SimpleDirectoryReader(input_dir=pdf_directory, recursive=True)
    documents = loader.load_data(num_workers=3)

    embed_model = HuggingFaceEmbedding(model_name=text_embedding_model)
    llm = LlamaCPP(
        model_url=llm_url,
        temperature=0.5,
        max_new_tokens=256,
        context_window=4096,
        generate_kwargs={"stop": ["<s>", "[INST]", "[/INST]"]},
        model_kwargs={"n_gpu_layers": -1},
        verbose=True
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
    return index, llm

# Define a function to filter context based on a specified keyword
def filter_context(context, start_keyword):
    start_index = context.find(start_keyword)
    if start_index != -1:
        return context[start_index:]
    return context  # Return the original context if the keyword is not found

# Define a function to clean and filter text
def clean_text(text):
    cleaned_text = text.replace("<pad>", "").replace("<EOS>", "").replace("\n", " ")
    cleaned_text = " ".join(cleaned_text.split())
    cleaned_text = filter_context(cleaned_text, "Figure 3")
    return cleaned_text

# Load custom CSS
st.markdown('<style>' + open('style.css').read() + '</style>', unsafe_allow_html=True)

# Streamlit app
st.markdown('<h2>Document Retrieval using RAG for Impetus Technologies</h2>', unsafe_allow_html=True)

pdf_directory = '/content/sample_data'
text_embedding_model = 'thenlper/gte-base'
llm_url = 'https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'

# Upload files
uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)
if uploaded_files:
    os.makedirs(pdf_directory, exist_ok=True)
    for uploaded_file in uploaded_files:
        file_path = os.path.join(pdf_directory, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
    st.success("Files uploaded successfully")

    # Verify files in the directory
    files_in_directory = os.listdir(pdf_directory)
    #st.write("Files in the directory:", files_in_directory)

    # Check if files were saved
    if not files_in_directory:
        st.error("No files found in the directory. Please check the upload.")
    else:
        # Process the uploaded files
        st.write("Processing documents...")
        start_time = time.time()
        index, llm = load_and_process_pdf(pdf_directory, text_embedding_model, llm_url)
        index_creation_time = time.time() - start_time
        #st.write(f"Time taken to create the index: {index_creation_time:.2f} seconds")

        # Prompt Template (RAG)
        concise_template = Prompt("""
        Based on the following context from the document, providing a concise response to the user's query:
        Document Context:
        {context_str}
        User Query:
        {query_str}

        Response (one paragraph):
        """)

        # Query Engine
        query_engine = index.as_query_engine(streaming=True, llm=llm)

        # Chat UI
        conversation_history = "Hi. [INST] How can I assist you with the content of the document today?"
        st.write("Start querying the document:")
        user_query = st.text_input("Enter your query")

        if st.button("Submit Query"):
            conversation_history += user_query + " [/INST] "

            response = query_engine.query(user_query)
            st.write("Document Context Retrieved:")
            for source_document in response.source_nodes:
                cleanText = clean_text(source_document.node.get_text())
                st.write(cleanText)

            conversation_history += response.response_text + " [/INST] "
            st.write(f"Response: {response.response_text}")

# Add a footer
st.markdown('<footer><p>Streamlit app by Anushree Sharma</p></footer>', unsafe_allow_html=True)


Overwriting app.py


In [12]:
import os
from pyngrok import ngrok
import streamlit as st
import llama_index
from pyngrok import ngrok
import subprocess

# Set the authtoken
authtoken = '2j9MNrMbeZEpvOcfinlFoOaatLS_7qwJiUjjX4P1tPGgrcb7P'
ngrok.set_auth_token(authtoken)

# Run the Streamlit app
os.system('streamlit run app.py &')

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")
# Run Streamlit app
subprocess.run(["streamlit", "run", "app.py"])



Streamlit app is live at: NgrokTunnel: "https://7d23-34-168-20-161.ngrok-free.app" -> "http://localhost:8501"


KeyboardInterrupt: 